# Settings for Download: Landsat

[Dataset](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2)

In [4]:
# ADJUST ONLY THESE PARAMETERS!

# If loop broke, resume by setting this i to where you want to continue
skip_to_i = 0 # 0 = no skip

# Timescale of Interest
my_plus_years   = 5 # Years to add after first visit
my_minus_years  = 2 # Years to add before first visit
my_first_date   = "-01-01"  # First date of first year "-MM-DD"
my_last_date    = "-12-31"  # Last date of last year "-MM-DD"

# Dataset of Interest
product            = "LANDSAT/LC08/C02/T1_L2"
product_start_date = "2013-03-18"
product_end_date   = "2023-10-13"
product_scale      = 700

output_scale       = product_scale
output_folder      = "gee-raw-data/landsat"

my_bands           = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7", "QA_PIXEL"]

# Download Routine Below

In [12]:
%run -i gee_functions.py

In [6]:
# Import Libraries
import ee
# ee.Authenticate()
ee.Initialize()

import os, re
import pandas as pd
from gee_subset import gee_subset
import geopandas as gpd
from datetime import datetime

In [7]:
data = get_location_site_df()
data_clean = adjust_first_last_date(
    df = data,
    plus_years  = my_plus_years,
    minus_years = my_minus_years,
    first_date  = my_first_date,
    last_date   = my_last_date,
)

cnt     = len(data_clean)
siteSet = list(range(0, cnt, 1))

In [8]:
# data_clean.head(10)

In [9]:
# print("Acess via [0, 1]: ", data_clean.columns[1], "\t = ", data_clean.iloc[0, 1])
# print("Acess via [0, 2]: ", data_clean.columns[2], "\t = ", data_clean.iloc[0, 2])
# print("Acess via [0, 3]: ", data_clean.columns[3], "\t = ", data_clean.iloc[0, 3])
# print("Acess via [0, 4]: ", data_clean.columns[4], "\t = ", data_clean.iloc[0, 4])

In [10]:
# Create folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [ ]:
download_gee_data(
    siteSet,
    skip_to_i,
    product,
    my_bands,
    data_clean,
    product_start_date,
    product_scale,
    output_scale,
    output_folder
)